<h1> Parameter Tunning 
    
    Look at different steps & batch sizes (12 months of data used)

# Importing libraries:

In [59]:
!pip install pandas
!pip install sklearn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import data; fill nan values; split data intro train and test:

In [60]:
# import data
data = pd.read_csv('Data_set_1_smaller.csv', index_col = 0)

# for later use
features_num = 15

# 2018 data
data = data.loc[data.index > 2018000000, :]

# reset index
data.reset_index(inplace = True)
data.drop('index', axis = 1, inplace = True)

# fill nan values
data.fillna(method = 'ffill', inplace = True)

from sklearn.model_selection import train_test_split

# divide data into train and test 
data_train, data_test = train_test_split(
         data, test_size = 0.15, shuffle=False)

# Apply feature scaling:

In [61]:
from sklearn.preprocessing import MinMaxScaler

# data scaling  (including offer (y))
sc_X = MinMaxScaler()
data_train = sc_X.fit_transform(data_train)
data_test = sc_X.transform(data_test)

# Importing the Keras libraries and packages:

In [62]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras import initializers
from keras import optimizers
from keras.callbacks import EarlyStopping

mae_cv = []
mse_cv = []
mae_gen = []
mse_gen  =[]
rmse_gen = []
mae_nor = []
mae_spi = []
mse_nor = []
mse_spi = []
rmse_nor = []
rmse_spi = []
prediction_list = []
time_count = []

# Prepare of data according to LSTM needs,  create regressor & tune:

In [63]:
# parameters
# steps = 96
# n_hidden = 1
units = 20
# batch_size = 96

from sklearn.model_selection import ParameterGrid
parameters = {'steps': [48, 96, 144, 336],
              'batch_size': [100, 150, 200, 250]}

all_param = ParameterGrid(parameters)

# parameters
# steps = 100
n_hidden = 1
units = 20
# batch_size = 100

# function to split data into correct shape for RNN
def split_data(X, y, steps):
    X_, y_ = list(), list()
    for i in range(steps, len(y)):
        X_.append(X[i - steps : i, :])
        y_.append(y[i]) 
    return np.array(X_), np.array(y_)

# function to cut data set so it can be divisible by the batch_size
def cut_data(data, batch_size):
     # see if it is divisivel
    condition = data.shape[0] % batch_size
    if condition == 0:
        return data
    else:
        return data[: -condition]

# divide features and labels
X_train = data_train[:, 0:15] 
y_train = data_train[:, -1]
X_test = data_test[:, 0:15] 
y_test = data_test[:, -1] 

# divide data into train and test 
X_train, X_val, y_train, y_val = train_test_split(
         X_train, y_train, test_size = 0.15, shuffle=False)

# put data into correct shape
X_train, y_train = split_data(X_train, y_train, steps)
X_test, y_test = split_data(X_test, y_test, steps)
X_val, y_val = split_data(X_val, y_val, steps)

X_train = cut_data(X_train, batch_size)
y_train = cut_data(y_train, batch_size)
X_test = cut_data(X_test, batch_size)
y_test = cut_data(y_test, batch_size)
X_val = cut_data(X_val, batch_size)
y_val = cut_data(y_val, batch_size)

for i in range(len(all_param)):
    
    start_time = time.time()
    
    # set parameters right
    steps = all_param[i]['steps']
    batch_size = all_param[i]['batch_size']
    
    # design the LSTM
    def regressor_tunning(kernel_initializer = 'he_normal',
                          bias_initializer = initializers.Ones()):
        model = Sequential()
        model.add(LSTM(units = units,                    
                       batch_input_shape = (batch_size, steps, features_num), 
                       stateful = True,
                       return_sequences = True,
                       kernel_initializer = kernel_initializer,
                       bias_initializer = bias_initializer))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dropout(0.2))
        for layer in list(reversed(range(n_hidden))):
            if layer == 0:
                model.add(LSTM(units = units, 
                               kernel_initializer = kernel_initializer,
                               bias_initializer = bias_initializer))
                model.add(LeakyReLU(alpha = 0.2))
                model.add(Dropout(0.2))
            else:
                model.add(LSTM(units = units, 
                               batch_input_shape = (batch_size, steps, features_num), 
                               stateful = True,
                               return_sequences = True,
                               kernel_initializer = kernel_initializer,
                               bias_initializer = bias_initializer))
                model.add(LeakyReLU(alpha = 0.2))
                model.add(Dropout(0.2))
        model.add(Dense(1, activation='linear'))
        model.compile(loss = 'mse', metrics = ['mse', 'mae'], optimizer = 'Adamax')
        return model

    model = regressor_tunning()

    # apply patience callback
    early_stopping = EarlyStopping(monitor='val_mse', patience=10)

    # fitting the LSTM to the training set
    history = model.fit(X_train,
                        y_train, 
                        batch_size = batch_size, 
                        epochs = i,
                        shuffle = False, 
                        validation_data = (X_val, y_val),
                        callbacks = early_stopping)

    # make new predicitons with test set
    y_pred = model.predict(X_test, batch_size = batch_size)

    # prices col = 15
    y_pred = (y_pred * sc_X.data_range_[15]) + (sc_X.data_min_[15])
    y_test = (y_test * sc_X.data_range_[15]) + (sc_X.data_min_[15])

    from sklearn.metrics import mean_squared_error as mse
    from sklearn.metrics import mean_absolute_error as mae

    rmse_error = mse(y_test, y_pred, squared = False)
    mse_error = mse(y_test, y_pred) # 1479.61335
    mae_error = mae(y_test, y_pred) # 23.1525

    rmse_gen.append(rmse_error)
    mse_gen.append(mse_error)
    mae_gen.append(mae_error)

    # =============================================================================
    # Metrics evaluation on spike regions
    # =============================================================================

    y_spike_occ = pd.read_csv('Spike_binary_1std.csv', usecols = [6])

    # create array same size as y_test
    y_spike_occ = y_spike_occ.iloc[- len(y_test):]
    y_spike_occ = pd.Series(y_spike_occ.iloc[:,0]).values


    # smal adjustment
    y_test = pd.Series(y_test)
    y_test.replace(0, 0.0001,inplace = True)


    # select y_pred and y_test only for regions with spikes
    y_test_spike = (y_test.T * y_spike_occ).T
    y_pred_spike = (y_pred.T * y_spike_occ).T
    y_test_spike = y_test_spike[y_test_spike != 0]
    y_pred_spike = y_pred_spike[y_pred_spike != 0]

    # calculate metric
    rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
    mse_spike = mse(y_test_spike, y_pred_spike)
    mae_spike = mae(y_test_spike, y_pred_spike)

    rmse_spi.append(rmse_spike)
    mse_spi.append(mse_spike)
    mae_spi.append(mae_spike)

    # =============================================================================
    # Metric evaluation on normal regions
    # =============================================================================

    # inverse y_spike_occ so the only normal occurences are chosen
    y_normal_occ = (y_spike_occ - 1) * (-1)

    # sanity check
    y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 

    # select y_pred and y_test only for normal regions
    y_test_normal = (y_test.T * y_normal_occ).T
    y_pred_normal = (y_pred.T * y_normal_occ).T
    y_test_normal = y_test_normal[y_test_normal != 0.00]
    y_pred_normal = y_pred_normal[y_pred_normal != 0.00]

    # calculate metric
    rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
    mse_normal = mse(y_test_normal, y_pred_normal)
    mae_normal = mae(y_test_normal, y_pred_normal)

    rmse_nor.append(rmse_normal)
    mse_nor.append(mse_normal)
    mae_nor.append(mae_normal)

    elapsed_time = time.time() - start_time

    time_count.append(elapsed_time)


Epoch 1/100
52/52 [==============================] - 3s 50ms/step - loss: 14800.1758 - mse: 14800.1758 - mae: 113.7615 - val_loss: 15740.8203 - val_mse: 15740.8203 - val_mae: 114.3649
Epoch 2/100
52/52 [==============================] - 1s 28ms/step - loss: 14764.0586 - mse: 14764.0586 - mae: 113.6032 - val_loss: 15705.3945 - val_mse: 15705.3945 - val_mae: 114.2099
Epoch 3/100
52/52 [==============================] - 1s 27ms/step - loss: 14728.8711 - mse: 14728.8711 - mae: 113.4489 - val_loss: 15670.9561 - val_mse: 15670.9561 - val_mae: 114.0590
Epoch 4/100
52/52 [==============================] - 1s 27ms/step - loss: 14694.6035 - mse: 14694.6045 - mae: 113.2971 - val_loss: 15636.6240 - val_mse: 15636.6240 - val_mae: 113.9084
Epoch 5/100
52/52 [==============================] - 1s 26ms/step - loss: 14659.5264 - mse: 14659.5264 - mae: 113.1434 - val_loss: 15601.9307 - val_mse: 15601.9307 - val_mae: 113.7560
Epoch 6/100
52/52 [==============================] - 1s 27ms/step - loss: 14625.

52/52 [==============================] - 2s 31ms/step - loss: 11699.7529 - mse: 11699.7529 - mae: 99.1564 - val_loss: 12659.5234 - val_mse: 12659.5234 - val_mae: 99.9543
Epoch 90/100
52/52 [==============================] - 1s 28ms/step - loss: 11659.3857 - mse: 11659.3857 - mae: 98.9504 - val_loss: 12624.5781 - val_mse: 12624.5781 - val_mae: 99.7783
Epoch 91/100
52/52 [==============================] - 1s 27ms/step - loss: 11623.6641 - mse: 11623.6641 - mae: 98.7966 - val_loss: 12590.4668 - val_mse: 12590.4668 - val_mae: 99.6062
Epoch 92/100
52/52 [==============================] - 1s 25ms/step - loss: 11588.2949 - mse: 11588.2949 - mae: 98.5889 - val_loss: 12556.6592 - val_mse: 12556.6602 - val_mae: 99.4354
Epoch 93/100
52/52 [==============================] - 1s 26ms/step - loss: 11559.1289 - mse: 11559.1289 - mae: 98.4846 - val_loss: 12522.8828 - val_mse: 12522.8828 - val_mae: 99.2648
Epoch 94/100
52/52 [==============================] - 1s 27ms/step - loss: 11529.2920 - mse: 11529

49/49 [==============================] - 9s 191ms/step - loss: 13390.2773 - mse: 13390.2783 - mae: 107.1183 - val_loss: 14498.1436 - val_mse: 14498.1445 - val_mae: 107.6000
Epoch 35/100
49/49 [==============================] - 9s 194ms/step - loss: 13345.7588 - mse: 13345.7588 - mae: 106.9210 - val_loss: 14460.5166 - val_mse: 14460.5166 - val_mae: 107.4251
Epoch 36/100
49/49 [==============================] - 10s 207ms/step - loss: 13317.7773 - mse: 13317.7773 - mae: 106.7734 - val_loss: 14422.7881 - val_mse: 14422.7881 - val_mae: 107.2491
Epoch 37/100
49/49 [==============================] - 10s 204ms/step - loss: 13271.9375 - mse: 13271.9375 - mae: 106.5671 - val_loss: 14384.9717 - val_mse: 14384.9717 - val_mae: 107.0729
Epoch 38/100
49/49 [==============================] - 9s 192ms/step - loss: 13233.2695 - mse: 13233.2695 - mae: 106.3822 - val_loss: 14346.8281 - val_mse: 14346.8281 - val_mae: 106.8953
Epoch 39/100
49/49 [==============================] - 9s 187ms/step - loss: 13193

34/34 [==============================] - 1s 29ms/step - loss: 13937.1934 - mse: 13937.1934 - mae: 109.8535 - val_loss: 15043.0967 - val_mse: 15043.0967 - val_mae: 110.9098
Epoch 43/100
34/34 [==============================] - 1s 41ms/step - loss: 13914.4756 - mse: 13914.4756 - mae: 109.7539 - val_loss: 15020.6934 - val_mse: 15020.6934 - val_mae: 110.8083
Epoch 44/100
34/34 [==============================] - 1s 41ms/step - loss: 13893.8311 - mse: 13893.8311 - mae: 109.6637 - val_loss: 14999.0615 - val_mse: 14999.0615 - val_mae: 110.7109
Epoch 45/100
34/34 [==============================] - 1s 43ms/step - loss: 13876.7139 - mse: 13876.7139 - mae: 109.5748 - val_loss: 14977.5459 - val_mse: 14977.5459 - val_mae: 110.6141
Epoch 46/100
34/34 [==============================] - 1s 35ms/step - loss: 13853.9404 - mse: 13853.9385 - mae: 109.4809 - val_loss: 14955.8076 - val_mse: 14955.8076 - val_mae: 110.5163
Epoch 47/100
34/34 [==============================] - 1s 32ms/step - loss: 13828.6035 - 

KeyboardInterrupt: 

# Save results into a data frame

In [ ]:
# Save
results = pd.DataFrame({'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor,
                       
                        'time': time_count})


In [ ]:
def highlight_min(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_max]

results.style.apply(highlight_min)

In [ ]:
# Save
results = pd.DataFrame({'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor,
                       
                        'time': time_count}, index = all_param)
results

In [ ]:
!pip install matplotlib

%matplotlib notebook

import matplotlib.pyplot as plt

In [ ]:
plt.plot(y_pred, label = 'predicted value')
plt.plot(y_test, label = 'actual value')
plt.legend()